# Optimizing our Parameters

### Introduction

In this lesson, we'll build out the component that finds the hypothesis function that minimizes the output of our loss component. 

### Loading our classes

To do so we'll need to create some Hypothesis instances.  Copy and paste the Hypothesis class from the previous lab here.

In [1]:
# Hypothesis
class Hypothesis:
    def __init__(self, coef_, intercept_, x_values):
        self.coef_ = coef_
        self.intercept_ = intercept_
        self.x_values = x_values
    
    def predict_value(self, input_value):
        return self.coef_*input_value + self.intercept_
    
    def predict(self):
        return [self.predict_value(x_value) for x_value in self.x_values]
    
    def trace(self, mode = 'lines', name=None, text = []):
        coef_text = f"y = {self.coef_}x"
        intercept_text = f" + {self.intercept_}"
        default_text = coef_text + intercept_text if self.intercept_ else coef_text
        text = name or default_text
        return {'x': self.x_values, 'y': self.predict(), 'mode': mode, 'name': name, 'text': text}

### Creating our Optimizer Class

This time, we left in the beginning components to our Optimizer class.  

Let's take a look at the `__init__` function.  Note that the optimizer begins by taking in our data of the actual `x_values` and `y_values`.  It also takes in the y-intercept value.  This is because we will not tackle the more complicated problem of having our Optimizer find both parameters, it will only find the coefficient.

The `start`, `stop` and `step_count` parameters will be explained further down below.  So will the `steps` function. 

In [22]:
class Optimizer:
    def __init__(self, x_values, y_values, intercept, start, stop, step_count):
        self.x_values = x_values
        self.y_values = y_values
        self.intercept = intercept
        self.start = start
        self.stop = stop
        self.step_count = step_count
        
    def steps(self):
        step_size = (self.stop - self.start)/self.step_count
        self.steps = []
        for count in list(range(0, self.step_count)):
            self.steps.append(self.start + count*step_size)
        return self.steps
    
    def set_hypotheses(self):
        coefs = self.steps
        self.hypotheses = [Hypothesis(coef, self.intercept, self.x_values) for coef in coefs]
    
    def set_losses(self):
        self.losses = [Loss(hypothesis, self.x_values, self.y_values) for hypothesis in self.hypotheses]
        
    def find_min(self):
        minimum_loss = self.losses[0]
        for loss in self.losses[1:]:
            if loss.rss() < minimum_loss.rss():
                minimum_loss = loss
        return minimum_loss

Now the goal of our optimizer is to find the value for our coefficient parameter that minimize the our `rss`.  The way that we'll do this is to create a list of Hypothesis instances, each with a sequential value of `m`.  

So our first Hypothesis could be.

In [23]:
intercept = 153
x_values = [800, 1500, 2000, 3500, 4000]
coef = .01

hyp = Hypothesis(coef, intercept, x_values)

And the second Hypothesis instance would have the same data except a slightly different coefficient parameter.

In [24]:
coef = .02
hyp = Hypothesis(coef, intercept, x_values)

The plan is to create a list of these Hypothesis instances, and then use our Loss class to find the hypothesis with the smallest rss score.

So that's where our steps method enters the picture: it generates a list of $m$ values to then pass through to create a list of Hypothesis instances.  To do so we just need to tell our Optimizer of a starting point, a stopping point, and a step size.  Let's see this in action.

In [25]:
intercept = 153
coef = .01
start = 0
stop = 1
step_number = 100
x_values = [800, 1500, 2000, 3500, 4000]
outcomes = [330, 780, 1130, 1310, 1780]
optimizer = Optimizer(x_values, outcomes, intercept, start, stop, step_number)

In [26]:
optimizer.__dict__

{'x_values': [800, 1500, 2000, 3500, 4000],
 'y_values': [330, 780, 1130, 1310, 1780],
 'intercept': 153,
 'start': 0,
 'stop': 1,
 'step_count': 100}

So now we have a list of steps.  Each one of these could be a different value for `m`.

In [27]:
optimizer.steps()[0:10]
# [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]

[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]

Let's use our steps to create a number of Hypothesis instances, each with the same input values and y-intercept, but a different coefficient.  If you look at the `set_hypotheses` step, you'll that it is responsible for creating a list of `hypotheses`.

In [28]:
optimizer.set_hypotheses()

Now we have a list of hypotheses, each with a different coefficient.

In [29]:
optimizer.hypotheses[0:3]
# [<__main__.Hypothesis at 0x10bbe12b0>,
#  <__main__.Hypothesis at 0x10bbe1630>,
#  <__main__.Hypothesis at 0x10bbe1470>]


In [30]:
[hyp.coef_ for hyp in optimizer.hypotheses[0:9]]
# [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]

[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]

### Your Task

So now we have an optimizer class, that can create a number of Hypotheses instances, each with a different coefficient.  What's left is to calculate the `rss` for each of these Hypotheses instances, and find the instance with the lowest `rss`.  

Our hypotheses instances don't have the capability to calculate the `rss`.  That functionality lies with our `Loss` instances.  First copy and paste the Loss class below.

In [31]:
# Loss class
class Loss():
    def __init__(self, hypothesis, x_values, y_values):
        self.hypothesis = hypothesis
        self.x_values = x_values
        self.y_values = y_values
        
    def errors(self):
        predicted_values = self.hypothesis.predict()
        zipped_list = zip(self.y_values,predicted_values)
        return [actual-expected for actual,expected in zipped_list]
    
    def squared_errors(self):
        error_list = self.errors()
        return [error**2 for error in error_list]
    
    def rss(self):
        squared_error_list = self.squared_errors()
        return sum(squared_error_list)

 So in the `set_losses` method, use each of the hypothesis instances to create a list of Loss instances, and assign these loss instances to an attribute called `losses`.

In [32]:
optimizer.set_losses()
optimizer.losses[0:3]

# [<__main__.Loss at 0x10bc185c0>,
#  <__main__.Loss at 0x10bc185f8>,
#  <__main__.Loss at 0x10bc18630>]

Each of the losses should store the related hypothesis instances and should also store the x_values and y_values of the optimizer.

In [33]:
optimizer.losses[0].__dict__

# {'hypothesis': <__main__.Hypothesis at 0x10bbe12b0>,
#  'x_values': [800, 1500, 2000, 3500, 4000],
#  'y_values': [330, 780, 1130, 1310, 1780]}

{'hypothesis': <__main__.Hypothesis at 0x10ad17ef0>,
 'x_values': [800, 1500, 2000, 3500, 4000],
 'y_values': [330, 780, 1130, 1310, 1780]}

In [34]:
[loss.hypothesis.coef_ for loss in optimizer.losses[0:9]]

# [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]

[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]

### Find the minimum

Now write a method called `find_min` that returns the Loss object with the lowest `rss`.  From there, we can find the related Hypothesis instance, and it's parameters.

In [35]:
loss = optimizer.find_min()
loss.hypothesis.__dict__

# {'coef_': 0.39, 'intercept_': 153, 'x_values': [800, 1500, 2000, 3500, 4000]}

{'coef_': 0.39, 'intercept_': 153, 'x_values': [800, 1500, 2000, 3500, 4000]}

We can see that the hypothesis that minimizes our RSS the hypothesis with a coefficient of .39.  This is within one one-hundredth of what we calculated with SKLearn.

### Summary

In this lesson, we'll build out the component that finds the hypothesis function that minimizes the output of our loss component.  